# 02. RSI Calculation
## Rapid Spread Index Integration

![SYLVA](https://img.shields.io/badge/SYLVA-v1.0.0-blue)

This notebook demonstrates the calculation of Rapid Spread Index (RSI) from nine normalized parameters.

In [ ]:
# Import libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath('..'))

from sylva_fire.integration.rsi_calculator import RSICalculator
from sylva_fire.forecasting.rapid_spread_forecast import RapidSpreadForecaster

print("✅ Libraries imported")

## RSI Equation

$$RSI = \sum_{i=1}^{9} \alpha_i \cdot P_i^{norm}$$

Where:
- αᵢ: Fuel type-specific weights
- Pᵢⁿᵒʳᵐ: Normalized parameters (0-1 scale)

In [ ]:
# Initialize RSI calculator for different fuel types
fuel_types = ['pinus_halepensis', 'quercus_ilex', 'mediterranean_maquis', 'dry_grassland']
calculators = {ft: RSICalculator(ft) for ft in fuel_types}

# Display weights for each fuel type
weights_df = pd.DataFrame()
for ft, calc in calculators.items():
    weights_df[ft] = pd.Series(calc.weights)

print("\n=== RSI Weights by Fuel Type ===\n")
print(weights_df.round(3))

In [ ]:
# Example parameters from Mati 2018 fire
params = {
    'lfm': 65,
    'dfm': 6,
    'cbd': 0.18,
    'sfl': 45,
    'fbd': 0.8,
    'wind_speed': 10.4,
    'vpd': 38.1,
    'aspect': 225,
    'drought_code': 487
}

print("=== Input Parameters ===")
for key, value in params.items():
    print(f"{key:12s}: {value}")

In [ ]:
# Calculate normalized values and RSI for each fuel type
results = []

for ft in fuel_types:
    calc = calculators[ft]
    normalized = calc.normalize_parameters(params)
    rsi = calc.calculate_rsi(normalized)
    contributions = calc.get_parameter_contributions(normalized)
    
    results.append({
        'Fuel Type': ft,
        'RSI': rsi,
        **{f'norm_{k}': v for k, v in normalized.items()}
    })
    
    print(f"\n=== {ft} ===")
    print(f"RSI: {rsi:.3f}")
    print("\nParameter Contributions:")
    for param, contrib in sorted(contributions.items(), key=lambda x: x[1], reverse=True):
        print(f"  {param}: {contrib:.1%}")

In [ ]:
# Visualize RSI comparison
plt.figure(figsize=(10, 6))
rsi_values = [r['RSI'] for r in results]
bars = plt.bar(fuel_types, rsi_values, color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D'])

# Add value labels
for bar, val in zip(bars, rsi_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{val:.3f}', ha='center', va='bottom')

plt.axhline(y=0.6, color='r', linestyle='--', label='Warning threshold')
plt.axhline(y=0.8, color='darkred', linestyle='--', label='Imminent threshold')
plt.ylabel('Rapid Spread Index (RSI)')
plt.title('RSI Comparison Across Fuel Types')
plt.ylim(0, 1)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.show()

In [ ]:
# Spider plot of normalized parameters
from math import pi

calc = calculators['pinus_halepensis']
normalized = calc.normalize_parameters(params)

categories = list(normalized.keys())
values = list(normalized.values())
N = len(categories)

# Create spider plot
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
values += values[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
plt.xticks(angles[:-1], categories, size=12)
ax.plot(angles, values, 'o-', linewidth=2, color='#2E86AB')
ax.fill(angles, values, alpha=0.25, color='#2E86AB')
ax.set_ylim(0, 1)
plt.title('Normalized Parameters - Pinus halepensis', size=16, pad=20)
plt.show()

In [ ]:
# Sensitivity analysis
param_ranges = {
    'lfm': np.linspace(40, 120, 20),
    'dfm': np.linspace(4, 20, 20),
    'wind_speed': np.linspace(0, 20, 20)
}

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, (param, values) in enumerate(param_ranges.items()):
    rsi_values = []
    for val in values:
        test_params = params.copy()
        test_params[param] = val
        normalized = calc.normalize_parameters(test_params)
        rsi_values.append(calc.calculate_rsi(normalized))
    
    axes[idx].plot(values, rsi_values, 'b-', linewidth=2)
    axes[idx].set_xlabel(param.upper())
    axes[idx].set_ylabel('RSI')
    axes[idx].set_title(f'RSI Sensitivity to {param.upper()}')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
print("\n✅ RSI calculation completed successfully")